In [92]:
using Pkg
Pkg.activate("..")
using Dispersions
using Test
using FFTW
include("../test/helper_functions.jl")

  Activating project at `~/Codes/Dispersions.jl`


fcc_dispersion (generic function with 1 method)

In [180]:
Ns = 4


gr = "fcc-1.1"
kG = gen_kGrid(gr, Ns)

v_full = collect(Dispersions.gen_sampling(grid_type(kG), 2, Ns))
q0 = Tuple(findfirst(x-> all(x .≈ 0), v_full))

arr1 = randn(ComplexF64, gridshape(kG))
arr2 = randn(ComplexF64, gridshape(kG))
arr1_sym = expandKArr(kG, reduceKArr(kG, arr1))
arr2_sym = expandKArr(kG, reduceKArr(kG, arr2))
arr1_c = deepcopy(arr1_sym)
arr2_c = deepcopy(arr2_sym);

In [181]:
res_pp, dbg_i_pp, dbg_val_pp = naive_conv(v_full, v_full, pp=true, q0, round_entries=false);
res_ph, dbg_i, dbg_val = naive_conv(v_full, v_full, pp=false, q0, round_entries=false);

In [182]:
q_list = map(el->check_q(el, Ns), dbg_val);
# preprocess rounds all entries in order for unqiue to work
q_list_pre = map(x -> map(xi -> round.(xi,digits=8), x), q_list)
q_list_pre = map(x -> map(xi -> map(xii -> xii ≈ 0 ? abs(xii) : xii, xi), x), q_list_pre)
# Test if all entries have the same common q difference
@test all(map(qi_list -> length(unique(qi_list)) == 1, q_list_pre))
# Extract only q vector (since we checked that only one unique exists
q_list_single = map(x->x[1], q_list_pre)
# Test if all q-vectors are at the correct index
@test all(map(x-> all(x[1] .≈ x[2]), zip(q_list_single, v_full)))
# Test if all possible q-vectors are contained in convolution
@test all(map(el -> all(el[1] .≈ el[2]), zip(sort(unique(v_full)[:]), sort(map(qi->qi[1], q_list_pre)[:]))))
# Test if all combinations of k_i and k_{i+j} are contained in each result index
@test all(map(entry -> length(unique(map(x->x[1], entry))), dbg_val) .== Ns*Ns)
@test all(map(entry -> length(unique(map(x->x[2], entry))), dbg_val) .== Ns*Ns)

Test Failed at In[182]:10
  Expression: all(map((x->begin
                all(x[1] .≈ x[2])
            end), zip(q_list_single, v_full)))



LoadError: [91mThere was an error during testing[39m

In [183]:
q_list_pp = map(el->check_q(el, Ns, pp=true), dbg_val_pp);
# preprocess rounds all entries in order for unqiue to work
q_list_pp_pre = map(x -> map(xi -> round.(xi,digits=8), x), q_list_pp)
q_list_pp_pre = map(x -> map(xi -> map(xii -> xii ≈ 0 ? abs(xii) : xii, xi), x), q_list_pp_pre)
# Test if all entries have the same common q difference
@test all(map(qi_list -> length(unique(qi_list)) == 1, q_list_pp_pre))
# extract only q vector (since we checked that only one unique exists
q_list_pp_single = map(x->x[1], q_list_pp_pre)
# test if all q-vectors are at the correct index
@test all(map(x-> all(x[1] .≈ x[2]), zip(q_list_pp_single, v_full)))
# Test if all possible q-vectors are contained in convolution
@test all(map(el -> all(el[1] .≈ el[2]), zip(sort(unique(v_full)[:]), sort(map(qi->qi[1], q_list_pp_pre)[:]))))
# Test if all combinations of k_i and k_{i+j} are contained in each result index
@test all(map(entry -> length(unique(map(x->x[1], entry))), dbg_val_pp) .== Ns*Ns)
@test all(map(entry -> length(unique(map(x->x[2], entry))), dbg_val_pp) .== Ns*Ns)

Test Failed at In[183]:10
  Expression: all(map((x->begin
                all(x[1] .≈ x[2])
            end), zip(q_list_pp_single, v_full)))



LoadError: [91mThere was an error during testing[39m

In [184]:
conv_ph_naive,_,_ = naive_conv(arr1_sym, arr2_sym, q0, pp=false);
conv_test = Dispersions.conv(kG, reduceKArr(kG,arr1_sym), reduceKArr(kG,arr2_sym), crosscorrelation=true)
conv_ph_fft_test = circshift(reverse(ifft(fft(arr1_sym) .* fft(reverse(arr2_sym)))), q0 .- 1)

show(stdout, "text/plain", expandKArr(kG, conv_test))
println("\n===============================")
show(stdout, "text/plain", kG.cache1 ./ kG.Nk)
println("\n===============================")
show(stdout, "text/plain", conv_ph_naive ./ kG.Nk)
println("\n===============================")
show(stdout, "text/plain", conv_ph_fft_test ./ kG.Nk)

@test all(conv_ph_fft_test .≈ conv_ph_naive)
@test all(conv_ph_fft_test .≈ expandKArr(kG, conv_test) .* kG.Nk)

4×4×4 Array{ComplexF64, 3}:
[:, :, 1] =
 -0.277594+0.066713im    0.0486769+0.0175387im  0.0635162-0.17637im     0.0486769+0.0175387im
 0.0486769+0.0175387im  -0.0522334-0.164458im   0.0614679+0.0729431im   -0.117465-0.0423572im
 0.0635162-0.17637im     0.0614679+0.0729431im  0.0708339-0.0598174im   0.0614679+0.0729431im
 0.0486769+0.0175387im   -0.117465-0.0423572im  0.0614679+0.0729431im  -0.0522334-0.164458im

[:, :, 2] =
  0.0486769+0.0175387im  -0.0522334-0.164458im    0.0614679+0.0729431im  -0.117465-0.0423572im
 -0.0522334-0.164458im     0.192215+0.138441im   -0.0696182-0.0592459im  0.0551615+0.0589822im
  0.0614679+0.0729431im  -0.0696182-0.0592459im   0.0674762+0.0596842im  -0.109823-0.0448006im
  -0.117465-0.0423572im   0.0551615+0.0589822im   -0.109823-0.0448006im  0.0551615+0.0589822im

[:, :, 3] =
 0.0635162-0.17637im     0.0614679+0.0729431im  0.0708339-0.0598174im   0.0614679+0.0729431im
 0.0614679+0.0729431im  -0.0696182-0.0592459im  0.0674762+0.0596842im   -0.109823-0.0

Test Passed

In [185]:
conv_pp_naive,_,_ = naive_conv(arr1_sym, arr2_sym, q0, pp=true);
conv_pp_test = Dispersions.conv(kG, reduceKArr(kG,arr1_sym), reduceKArr(kG, arr2_sym), crosscorrelation=false)
conv_pp_fft_test = circshift(ifft(fft(arr1_sym) .* fft(arr2_sym)), -0 .* q0 .+ 2)

#show(stdout, "text/plain", expandKArr(kG, conv_pp_test))
#println("\n===============================")
#show(stdout, "text/plain", kG.cache1 ./ kG.Nk)
#println("\n===============================")
show(stdout, "text/plain", conv_pp_naive  ./ kG.Nk )
println("\n===============================")
show(stdout, "text/plain", conv_pp_fft_test ./ kG.Nk)

@test all(conv_pp_fft_test .≈ conv_pp_naive)
@test all(conv_pp_fft_test ./ kG.Nk .≈ expandKArr(kG, conv_pp_test))

4×4×4 Array{ComplexF64, 3}:
[:, :, 1] =
 -0.108997+0.0324847im   0.0674762+0.0596842im  0.0708339-0.0598174im   0.0674762+0.0596842im
 0.0674762+0.0596842im  -0.0696182-0.0592459im  0.0614679+0.0729431im   -0.109823-0.0448006im
 0.0708339-0.0598174im   0.0614679+0.0729431im  0.0635162-0.17637im     0.0614679+0.0729431im
 0.0674762+0.0596842im   -0.109823-0.0448006im  0.0614679+0.0729431im  -0.0696182-0.0592459im

[:, :, 2] =
  0.0674762+0.0596842im  -0.0696182-0.0592459im   0.0614679+0.0729431im  -0.109823-0.0448006im
 -0.0696182-0.0592459im    0.192215+0.138441im   -0.0522334-0.164458im   0.0551615+0.0589822im
  0.0614679+0.0729431im  -0.0522334-0.164458im    0.0486769+0.0175387im  -0.117465-0.0423572im
  -0.109823-0.0448006im   0.0551615+0.0589822im   -0.117465-0.0423572im  0.0551615+0.0589822im

[:, :, 3] =
 0.0708339-0.0598174im   0.0614679+0.0729431im  0.0635162-0.17637im     0.0614679+0.0729431im
 0.0614679+0.0729431im  -0.0522334-0.164458im   0.0486769+0.0175387im   -0.117465-0.

Test Passed

In [186]:
@test all(arr1_c .== arr1_sym)
@test all(arr2_c .== arr2_sym)

Test Passed

# k-k'

In [187]:
all(map(x-> all(x[1] .≈ x[2]) , zip(k_plus_q, k_plus_q2[:])))

true

In [188]:
v_full
show(stdout, "text/plain", v_full)

4×4×4 Array{Tuple{Float64, Float64, Float64}, 3}:
[:, :, 1] =
 (0.0, 0.0, 0.0)      (0.0, 1.5708, 0.0)      (0.0, 3.14159, 0.0)      (0.0, 4.71239, 0.0)
 (1.5708, 0.0, 0.0)   (1.5708, 1.5708, 0.0)   (1.5708, 3.14159, 0.0)   (1.5708, 4.71239, 0.0)
 (3.14159, 0.0, 0.0)  (3.14159, 1.5708, 0.0)  (3.14159, 3.14159, 0.0)  (3.14159, 4.71239, 0.0)
 (4.71239, 0.0, 0.0)  (4.71239, 1.5708, 0.0)  (4.71239, 3.14159, 0.0)  (4.71239, 4.71239, 0.0)

[:, :, 2] =
 (0.0, 0.0, 1.5708)      (0.0, 1.5708, 1.5708)      (0.0, 3.14159, 1.5708)      (0.0, 4.71239, 1.5708)
 (1.5708, 0.0, 1.5708)   (1.5708, 1.5708, 1.5708)   (1.5708, 3.14159, 1.5708)   (1.5708, 4.71239, 1.5708)
 (3.14159, 0.0, 1.5708)  (3.14159, 1.5708, 1.5708)  (3.14159, 3.14159, 1.5708)  (3.14159, 4.71239, 1.5708)
 (4.71239, 0.0, 1.5708)  (4.71239, 1.5708, 1.5708)  (4.71239, 3.14159, 1.5708)  (4.71239, 4.71239, 1.5708)

[:, :, 3] =
 (0.0, 0.0, 3.14159)      (0.0, 1.5708, 3.14159)      (0.0, 3.14159, 3.14159)      (0.0, 4.71239, 3.14159)
 (1.570

In [149]:
minus_v_full = map(x-> -1 .* x, v_full)
show(stdout, "text/plain", minus_v_full)

6×6 Matrix{Tuple{Float64, Float64}}:
 (2.0944, 2.0944)    (2.0944, 1.0472)    (2.0944, -0.0)    (2.0944, -1.0472)    (2.0944, -2.0944)    (2.0944, -3.14159)
 (1.0472, 2.0944)    (1.0472, 1.0472)    (1.0472, -0.0)    (1.0472, -1.0472)    (1.0472, -2.0944)    (1.0472, -3.14159)
 (-0.0, 2.0944)      (-0.0, 1.0472)      (-0.0, -0.0)      (-0.0, -1.0472)      (-0.0, -2.0944)      (-0.0, -3.14159)
 (-1.0472, 2.0944)   (-1.0472, 1.0472)   (-1.0472, -0.0)   (-1.0472, -1.0472)   (-1.0472, -2.0944)   (-1.0472, -3.14159)
 (-2.0944, 2.0944)   (-2.0944, 1.0472)   (-2.0944, -0.0)   (-2.0944, -1.0472)   (-2.0944, -2.0944)   (-2.0944, -3.14159)
 (-3.14159, 2.0944)  (-3.14159, 1.0472)  (-3.14159, -0.0)  (-3.14159, -1.0472)  (-3.14159, -2.0944)  (-3.14159, -3.14159)

In [138]:
shift_vec = 2 .* kG.k0 .- gridshape(kG) .- 1
minus_v_full2 = circshift(reverse(v_full), shift_vec)
show(stdout, "text/plain", minus_v_full2)

6×6 Matrix{Tuple{Float64, Float64}}:
 (2.0944, 2.0944)   (2.0944, 1.0472)   (2.0944, 0.0)   (2.0944, -1.0472)   (2.0944, -2.0944)   (2.0944, 3.14159)
 (1.0472, 2.0944)   (1.0472, 1.0472)   (1.0472, 0.0)   (1.0472, -1.0472)   (1.0472, -2.0944)   (1.0472, 3.14159)
 (0.0, 2.0944)      (0.0, 1.0472)      (0.0, 0.0)      (0.0, -1.0472)      (0.0, -2.0944)      (0.0, 3.14159)
 (-1.0472, 2.0944)  (-1.0472, 1.0472)  (-1.0472, 0.0)  (-1.0472, -1.0472)  (-1.0472, -2.0944)  (-1.0472, 3.14159)
 (-2.0944, 2.0944)  (-2.0944, 1.0472)  (-2.0944, 0.0)  (-2.0944, -1.0472)  (-2.0944, -2.0944)  (-2.0944, 3.14159)
 (3.14159, 2.0944)  (3.14159, 1.0472)  (3.14159, 0.0)  (3.14159, -1.0472)  (3.14159, -2.0944)  (3.14159, 3.14159)

In [179]:
transform_k_to_minus_k(k) = map(ki -> ki != 0 ? mod.(-1 .* ki .+ π .- 1/Nk(kG), 2π) .- π .+ 1/Nk(kG) : ki, k)
minus_vec_naive = map(transform_k_to_minus_k, v_full)
@test all(map(x-> all(x[1] .≈ x[2]), zip(minus_vec_naive, minus_v_full2)))

Test Passed

In [163]:

show(stdout, "text/plain", minus_v_full2)

6×6 Matrix{Tuple{Float64, Float64}}:
 (2.0944, 2.0944)   (2.0944, 1.0472)   (2.0944, 0.0)   (2.0944, -1.0472)   (2.0944, -2.0944)   (2.0944, 3.14159)
 (1.0472, 2.0944)   (1.0472, 1.0472)   (1.0472, 0.0)   (1.0472, -1.0472)   (1.0472, -2.0944)   (1.0472, 3.14159)
 (0.0, 2.0944)      (0.0, 1.0472)      (0.0, 0.0)      (0.0, -1.0472)      (0.0, -2.0944)      (0.0, 3.14159)
 (-1.0472, 2.0944)  (-1.0472, 1.0472)  (-1.0472, 0.0)  (-1.0472, -1.0472)  (-1.0472, -2.0944)  (-1.0472, 3.14159)
 (-2.0944, 2.0944)  (-2.0944, 1.0472)  (-2.0944, 0.0)  (-2.0944, -1.0472)  (-2.0944, -2.0944)  (-2.0944, 3.14159)
 (3.14159, 2.0944)  (3.14159, 1.0472)  (3.14159, 0.0)  (3.14159, -1.0472)  (3.14159, -2.0944)  (3.14159, 3.14159)

In [166]:


show(stdout, "text/plain", tt)

6×6 Matrix{Tuple{Float64, Float64}}:
 (2.0944, 2.0944)       (2.0944, 1.0472)       (2.0944, 9.71445e-17)       (2.0944, -1.0472)       (2.0944, -2.0944)       (2.0944, 3.14159)
 (1.0472, 2.0944)       (1.0472, 1.0472)       (1.0472, 9.71445e-17)       (1.0472, -1.0472)       (1.0472, -2.0944)       (1.0472, 3.14159)
 (9.71445e-17, 2.0944)  (9.71445e-17, 1.0472)  (9.71445e-17, 9.71445e-17)  (9.71445e-17, -1.0472)  (9.71445e-17, -2.0944)  (9.71445e-17, 3.14159)
 (-1.0472, 2.0944)      (-1.0472, 1.0472)      (-1.0472, 9.71445e-17)      (-1.0472, -1.0472)      (-1.0472, -2.0944)      (-1.0472, 3.14159)
 (-2.0944, 2.0944)      (-2.0944, 1.0472)      (-2.0944, 9.71445e-17)      (-2.0944, -1.0472)      (-2.0944, -2.0944)      (-2.0944, 3.14159)
 (3.14159, 2.0944)      (3.14159, 1.0472)      (3.14159, 9.71445e-17)      (3.14159, -1.0472)      (3.14159, -2.0944)      (3.14159, 3.14159)